In [2]:
from datetime import datetime, date, time
import numpy as np
import pandas as pd

## Data ingestion

In [3]:
file_path = '../data'

In [4]:
df = pd.read_excel(f'{file_path}/Shipment-Waybills-Oct24.xlsx')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
df.columns

Index(['Waybill Date', 'Waybill', 'Account', 'Shipper', 'Consignee',
       'Orig Hub', 'Orig Place', 'Dest Hub', 'Dest Place', 'Pieces', 'Service',
       'Chrg Mass', 'Booking Date', 'Actual Transit Days', 'POD Date',
       'POD Recipient', 'SLA Transit Days', 'Due Date', 'Last Event',
       'Last Event Date'],
      dtype='object')

## Data cleaning

In [7]:
def safe_to_datetime(x):
    if pd.isna(x):
        return pd.NaT
    elif isinstance(x, (pd.Timestamp, np.datetime64, datetime)):
        return pd.to_datetime(x)
    elif isinstance(x, time):
        return pd.to_datetime(datetime.combine(date.today(), x))
    else:
        try:
            return pd.to_datetime(x)
        except:
            return pd.NaT

In [8]:
date_columns = ['Due Date', 'Waybill Date', 'Last Event Date']
for col in date_columns:
    df[col] = df[col].apply(safe_to_datetime)

## Data manipulation

In [9]:
# Build rules based on other columns

## Data output

In [11]:
account_list = df['Account'].unique()
for account in account_list:
    df_account = df[df['Account'] == account]
    df_account = df_account.sort_values(by=['Last Event', 'Waybill Date'], ascending=[True, False])

    date_columns = ['Due Date', 'Last Event Date', 'Waybill Date']
    for col in date_columns:
        df_account[col] = df_account[col].dt.strftime('%Y-%m-%d')

    df_account.to_excel(f'{file_path}/output/{account}.xlsx', index=False)